# Scratch

### Read TPCH Data

In [1]:
import duckdb
import pandas as pd
# No need to import duckdb_engine
#  jupysql will auto-detect the driver needed based on the connection string!

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql


In [2]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False


In [62]:
%sql duckdb:///:default:


Switching to connection duckdb:///:default:

In [63]:
sqlstr = """
DROP VIEW EMP;
CREATE VIEW EMP AS SELECT * FROM read_parquet("/Users/adarshrp/Projects/yard/data/emp.parquet");

DROP VIEW LINEITEM;
CREATE VIEW LINEITEM AS SELECT * FROM read_parquet("/Users/adarshrp/Downloads/part-0.parquet")"""

duckdb.sql(sqlstr)



In [64]:
%sql SELECT count(*) from EMP;

,count_star()
0,16


In [33]:
sqlstr = """
SELECT SUM(L_QUANTITY), COUNT(L_QUANTITY) AS SUM_QTY
FROM "/Users/adarshrp/Downloads/part-0.parquet"
GROUP BY L_RETURNFLAG;
"""

duckdb.sql(sqlstr)

┌─────────────────┬─────────┐
│ sum(L_QUANTITY) │ SUM_QTY │
│     double      │  int64  │
├─────────────────┼─────────┤
│       2469473.0 │   96913 │
│       2484642.0 │   97205 │
│       5074639.0 │  199098 │
└─────────────────┴─────────┘

In [ ]:
"""R""","""F""",2469473.0,"""F""","""F"""
"""A""","""F""",2484642.0,"""F""","""F"""
"""N""","""O""",5009601.0,"""O""","""O"""
"""N""","""F""",65038.0,"""F""","""F"""

In [41]:
sqlstr = """
SELECT MIN(L_SHIPDATE)
FROM "/Users/adarshrp/Downloads/part-0.parquet"
"""

duckdb.sql(sqlstr)

┌─────────────────┐
│ min(L_SHIPDATE) │
│      date       │
├─────────────────┤
│ 1992-01-02      │
└─────────────────┘

In [82]:
sqlstr = """
SET threads TO 10;

SELECT L_RETURNFLAG, L_LINESTATUS, SUM(L_QUANTITY) AS SUM_QTY,
SUM(L_EXTENDEDPRICE) AS SUM_BASE_PRICE, SUM(L_EXTENDEDPRICE*(-L_DISCOUNT + 1.0)) AS SUM_DISC_PRICE,
SUM(L_EXTENDEDPRICE*(-L_DISCOUNT + 1.0)*(L_TAX + 1.0)) AS SUM_CHARGE, AVG(L_QUANTITY) AS AVG_QTY,
AVG(L_EXTENDEDPRICE) AS AVG_PRICE, AVG(L_DISCOUNT) AS AVG_DISC, COUNT(L_QUANTITY) AS COUNT_ORDER
FROM LINEITEM
WHERE L_SHIPDATE <= '1998-12-01'
GROUP BY L_RETURNFLAG, L_LINESTATUS;
"""

duckdb.sql(sqlstr)

┌──────────────┬──────────────┬───────────┬───┬────────────────────┬──────────────────────┬─────────────┐
│ l_returnflag │ l_linestatus │  SUM_QTY  │ … │     AVG_PRICE      │       AVG_DISC       │ COUNT_ORDER │
│   varchar    │   varchar    │  double   │   │       double       │        double        │    int64    │
├──────────────┼──────────────┼───────────┼───┼────────────────────┼──────────────────────┼─────────────┤
│ N            │ O            │ 5009601.0 │ … │ 38211.636184674324 │  0.04991896140364839 │      196573 │
│ A            │ F            │ 2484642.0 │ … │  38360.56850604393 │ 0.049793940640911825 │       97205 │
│ R            │ F            │ 2469473.0 │ … │  38203.05635229532 │ 0.050062014384033425 │       96913 │
│ N            │ F            │   65038.0 │ … │   38841.9204079208 │  0.04975049504950487 │        2525 │
├──────────────┴──────────────┴───────────┴───┴────────────────────┴──────────────────────┴─────────────┤
│ 4 rows                                      

In [80]:
%sql select count(*) FROM LINEITEM 


,count_star()
0,393216


In [72]:
%sql select count(*) FROM LINEITEM  WHERE L_SHIPDATE <= '1998-12-01'

,count_star()
0,393216
